In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=4cff10ed01585f625954207bd87d71d1bb08b9a72be07dec2bc5907df1fe8a42
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F13 '''
def F13(X):
    f = bn.F13()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.04  ,random_state=0 , l1_ratio=1.0, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 40.53 , C = 980.40 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_13_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_13_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(20, 30):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i-20,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 20
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=314960, Sun Apr 19 02:15:16 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.014799748035447e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 9.619946558656986e+03 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10518.16589106] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10216.89501013] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 9.823332229501002e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   30    570 7.446206930506073e+03 1.1e+00 2.12e+00  2e+00  2e+00 0:03.4
   64   1216 7.455747764661746e+03 1.2e+00 2.05e+00  2e+00  2e+00 0:07.4
  100   1900 6.640853475837604e+03 1.2e+00 1.72e+00  2e+00  2e+00 0:11.0
  164   3116 5.258761769969378e+03 1.3e+00 1.02e+00  1e+00  1e+00 0:17.1
  200   3800 4.325295605172561e+03 1.3e+00 7.78e-01  8e-01  8e-01 0:20.5
  285   5415 3.145007032397602e+03 1.3e+00 3.18e-01  3e-01  3e-01 0:28.5
  300   5700 3.045639466941389e+03 1.3e+00 2.64e-01  3e-01  3e-01 0:30.0
  400   7600 2.839391961785998e+03 1.3e+00 9.36e-02  9e-02  9e-02 0:39.7
  500   9500 2.811658697787044e+03 1.3e+00 3.60e-02  4e-02  4e-02 0:50.2
  600  11400 2.807573358374385e+03 1.3e+00 1.27e-02  1e-02  1e-02 1:00.7
  700  13300 2.807073658003810e+03 1.3e+00 4.19e-03  4e-03  4e-03 1:12.2
  800  15200 2.807019646743143e+03 1.3e+00 1.43e-03  1e-03  1e-03 1:22.6
  900  17100 2.807013540810680e+03 1.3e+00 4.85e-04

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.69225751] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [11542.55090328] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.025864821048941e+04 1.0e+00 2.39e+00  2e+00  2e+00 0:00.2
    3     57 9.889391269525187e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


...

    3     57 9.750538377815601e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   32    608 7.329264881196780e+03 1.1e+00 2.03e+00  2e+00  2e+00 0:03.3
   71   1349 6.769182660606506e+03 1.2e+00 1.74e+00  2e+00  2e+00 0:07.4
  100   1900 5.936265650472344e+03 1.2e+00 1.31e+00  1e+00  1e+00 0:10.4
  158   3002 4.200703923587502e+03 1.2e+00 7.67e-01  8e-01  8e-01 0:16.5
  200   3800 3.528358745598554e+03 1.2e+00 4.66e-01  5e-01  5e-01 0:20.8
  277   5263 2.938684633059213e+03 1.2e+00 2.04e-01  2e-01  2e-01 0:28.9
  300   5700 2.890034583982539e+03 1.2e+00 1.61e-01  2e-01  2e-01 0:31.3
  397   7543 2.819293019724919e+03 1.2e+00 5.79e-02  6e-02  6e-02 0:41.4
  400   7600 2.818049049568122e+03 1.2e+00 5.59e-02  6e-02  6e-02 0:41.7
  500   9500 2.807986107845210e+03 1.2e+00 1.82e-02  2e-02  2e-02 0:52.1
  600  11400 2.807109879583604e+03 1.2e+00 5.72e-03  6e-03  6e-03 1:02.4
  700  13300 2.807024848381348e+03 1.2e+00 1.93e-03  2e-03  2e-03 1:12.8
  800  15200 2.807014089532824e+03 1.2e+00 6.73e-04

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10379.69225751] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [9871.73406935] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 9.370838002312430e+03 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 9.307117008191221e+03 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [10491.24142476] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   35    665 6.552280334172228e+03 1.1e+00 2.04e+00  2e+00  2e+00 0:03.4
   77   1463 6.335790810438410e+03 1.2e+00 1.85e+00  2e+00  2e+00 0:07.5
  100   1900 5.884850550281437e+03 1.2e+00 1.66e+00  2e+00  2e+00 0:09.7
  163   3097 4.708510450690073e+03 1.2e+00 1.03e+00  1e+00  1e+00 0:15.8
  200   3800 4.279555561072359e+03 1.3e+00 7.52e-01  7e-01  8e-01 0:19.3
  282   5358 3.862927457494476e+03 1.3e+00 4.25e-01  4e-01  4e-01 0:27.4
  300   5700 3.799342505225296e+03 1.3e+00 3.77e-01  4e-01  4e-01 0:29.1
  400   7600 3.644808385385673e+03 1.3e+00 1.77e-01  2e-01  2e-01 0:38.8
  482   9158 3.600067927239154e+03 1.3e+00 1.01e-01  1e-01  1e-01 0:49.9
  500   9500 3.598344664939736e+03 1.3e+00 9.46e-02  9e-02  1e-01 0:51.6
  600  11400 3.581275802645252e+03 1.3e+00 5.29e-02  5e-02  5e-02 1:01.2
  700  13300 3.576389234582077e+03 1.3e+00 2.96e-02  3e-02  3e-02 1:10.9
  800  15200 3.574035105689747e+03 1.5e+00 2.35e-02  2e-02  2e-02 1:20.6
  900  17100 3.572752004842562e+03 1.7e+00 1.71e-02

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('sample_data\\F13_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F13(X_Values.iloc[i,:200])
    SVM_Fun [i] = F13(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F13(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([2858.18445429]), count=array([1])) 2.3946138135337384e-05
SVM
ModeResult(mode=array([9600.49212373]), count=array([1])) 453.71878119950105
ELN
ModeResult(mode=array([7428.35338984]), count=array([1])) 19.216051238072286
